In [2]:
import numpy as np
import pandas as pd

In [3]:
class DataSource():
    def get_training_data(self):

        column_names = ['MPG', 'Cylinders', 'Displacement',
                        'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
        raw_dataset = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data",
                                  names=column_names, na_values="?", comment='\t', sep=" ", skipinitialspace=True)

        dataset = raw_dataset.copy()
        dataset = dataset.dropna()

        origin = dataset.pop('Origin')
        dataset['USA'] = (origin == 1)*1.0
        dataset['Europe'] = (origin == 2)*1.0
        dataset['Japan'] = (origin == 3)*1.0

        desc = dataset.describe()

        train_stats = dataset.describe()
        train_stats.pop("MPG")
        train_stats = train_stats.transpose()

        labels = dataset.pop('MPG')

        def norm(x):
            return (x - train_stats['mean']) / train_stats['std']

        normed_data = norm(dataset)

        X_raw = normed_data.to_numpy().T

        Y_raw = np.reshape(labels.to_numpy(), (1, len(labels)))

        X_train = X_raw[:, :-10]
        Y_train = Y_raw[:, :-10]
        X_test = X_raw[:, -10:]
        Y_test = Y_raw[:, -10:]

        return X_train, Y_train, X_test, Y_test, desc

In [4]:

class FourLayerModel:
    def __init__(self, input_features: int, hidden_layer1_neurons: int = 10, hidden_layer2_neurons: int = 5, output_layer_neurons: int = 1):
        """
        Arguments:
            input_features (int): The number of input features.
            hidden_layer1_neurons (int, optional): Number of Neurons for the first hidden layer. Defaults to 10.
            hidden_layer2_neurons (int, optional): Number of Neurons for the second hidden layer. Defaults to 5.
            output_layer_neurons (int, optional): Number of Neurons for the output layer. Defaults to 1.
        """
        np.random.seed(1)
        
        self.W1 = np.random.randn(hidden_layer1_neurons, input_features) * 0.01
        self.b1 = np.zeros((hidden_layer1_neurons, 1))

        self.W2 = np.random.randn(hidden_layer2_neurons, hidden_layer1_neurons) * 0.01
        self.b2 = np.zeros((hidden_layer2_neurons, 1))

        self.W3 = np.random.randn(output_layer_neurons, hidden_layer2_neurons) * 0.01
        self.b3 = np.zeros((output_layer_neurons, 1))

    def relu(self, Z):
        return np.maximum(Z, 0)

    def forward_propagation(self, X):
        """
        Argument:
        X -- Input data of shape (number of features, number of examples).

        Returns:
        The prediction of the model, of shape (1, number of examples)
        """

        # Calculate first hidden layer neurons
        self.Z1 = np.dot(self.W1, X) + self.b1
        # Calculate first hidden layer neuron activations
        self.A1 = self.relu(self.Z1)

        # Calculate second hidden layer neurons
        self.Z2 = np.dot(self.W2, self.A1) + self.b2
        # Calculate second hidden layer neuron activations
        self.A2 = self.relu(self.Z2)

        # Calculate output layer neurons
        self.Z3 = np.dot(self.W3, self.A2) + self.b3
        # Calculate output layer neuron activations
        self.A3 = self.Z3

        return self.A3

    def calculate_cost(self, prediction, Y):
        """
        Arguments:
        prediction -- The output of last activation, of shape (1, number of examples)
        Y -- "true" labels vector of shape (1, number of examples)

        Returns:
        cost -- computes the Mean Squared Error
        """

        return np.mean(np.square(Y - prediction))

    def relu_dev(self, X):
        return np.where(X > 0, 1, 0)

    def backward_propagation(self, X, Y):
        """
        Arguments:
        X -- Input data of shape (number of features, number of examples).
        Y -- "true" labels vector of shape (1, number of examples)
        """

        # Number of examples
        m = X.shape[1]

        # Mean Suared Error derivative
        self.dZ3 = -2 * (Y - self.A3)

        # Output layer derivatives
        self.dW3 = np.dot(self.dZ3, self.A2.T) / m
        self.db3 = np.sum(self.dZ3, axis=1, keepdims=True) / m

        # Second hidden layer derivatives
        self.dZ2 = np.dot(self.W3.T, self.dZ3) * self.relu_dev(self.A2)
        self.dW2 = np.dot(self.dZ2, self.A1.T) / m
        self.db2 = np.sum(self.dZ2, axis=1, keepdims=True) / m

        # First hidden layer derivatives
        self.dZ1 = np.dot(self.W2.T, self.dZ2) * self.relu_dev(self.A1)
        self.dW1 = np.dot(self.dZ1, X.T) / m
        self.db1 = np.sum(self.dZ1, axis=1, keepdims=True) / m

    def update_weights(self, learning_rate=0.01):
        """
        Argument:
        learning_rate -- The learning rate to apply in the weights update.
        """
        self.W1 = self.W1 - learning_rate * self.dW1
        self.b1 = self.b1 - learning_rate * self.db1
        self.W2 = self.W2 - learning_rate * self.dW2
        self.b2 = self.b2 - learning_rate * self.db2
        self.W3 = self.W3 - learning_rate * self.dW3
        self.b3 = self.b3 - learning_rate * self.db3


In [5]:
X_train, Y_train, X_test, Y_test, desc = DataSource().get_training_data()

In [6]:
TRAIN_EPOCHS = 15001
LEARNING_RATE = 0.001
PRINT_EACH = 1000

In [7]:
nn = FourLayerModel(X_train.shape[0], 5, 4)

In [8]:
for i in range(0, TRAIN_EPOCHS):
    predictions = nn.forward_propagation(X_train)
    cost = nn.calculate_cost(predictions, Y_train)

    nn.backward_propagation(X_train, Y_train)

    nn.update_weights(LEARNING_RATE)

    if (i) % PRINT_EACH == 0:
        print("Cost after iteration %i: %f " % (i, cost))

Cost after iteration 0: 601.210960 
Cost after iteration 1000: 7.917978 
Cost after iteration 2000: 7.495592 
Cost after iteration 3000: 6.847989 
Cost after iteration 4000: 6.512429 
Cost after iteration 5000: 6.316242 
Cost after iteration 6000: 6.199060 
Cost after iteration 7000: 6.104805 
Cost after iteration 8000: 6.063787 
Cost after iteration 9000: 6.036635 
Cost after iteration 10000: 6.019376 
Cost after iteration 11000: 6.007958 
Cost after iteration 12000: 5.997156 
Cost after iteration 13000: 5.990236 
Cost after iteration 14000: 5.984817 
Cost after iteration 15000: 5.979764 


In [9]:
test_predictions = nn.forward_propagation(X_test)

print("Predictions: ", test_predictions.astype(int))
print("Real Values: ", Y_test.astype(int))

Predictions:  [[29 26 30 32 25 28 42 34 29 28]]
Real Values:  [[26 22 32 36 27 27 44 32 28 31]]
